In [7]:
from deeprobust.graph.data import Dataset

from deeprobust.graph.utils import get_train_val_test
# loading cora dataset
data = Dataset(root='/tmp/', name='citeseer', seed=15)
adj, features, labels = data.adj, data.features, data.labels
# you can also split the data by yourself
# idx_train, idx_val, idx_test = get_train_val_test(adj.shape[0], val_size=0.1, test_size=0.8)

# loading acm dataset
#data = Dataset(root='/tmp/', name='acm', seed=15)

import torch
import numpy as np
#np.random.bit_generator = np.random._bit_generator
import torch.nn.functional as F
import torch.optim as optim
from deeprobust.graph.defense import GCN
from deeprobust.graph.defense import GAT
from deeprobust.graph.targeted_attack import RND
from deeprobust.graph.global_attack import DICE
from deeprobust.graph.utils import *
from tqdm import tqdm
import argparse
import scipy.sparse as sp
import scipy.io as sio

Loading citeseer dataset...


/anaconda3/envs/deeprobust/lib/python3.7/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:
/anaconda3/envs/deeprobust/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/anaconda3/envs/deeprobust/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):


Selecting 1 largest connected components


In [8]:
#idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
idx_train, idx_val, idx_test = get_train_val_test(adj.shape[0], val_size=0.2, test_size=0.2, stratify=data.labels)


idx_unlabeled = np.union1d(idx_val, idx_test)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
from deeprobust.graph.global_attack import PGDAttack
adj, features, labels = preprocess(adj, features, labels, preprocess_adj=False) 
gcn = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                        nhid=16, dropout=0.5, weight_decay=5e-4, device='cpu').to('cpu')
gcn.fit(features, adj, labels, idx_train)

data.idx_train = idx_train
data.idx_val = idx_val
data.idx_test = idx_test


In [ ]:
# Topology Attack
import tensorflow as tf
import torch

N=10
M_adj = torch.zeros((1,np.shape(adj)[0]*np.shape(adj)[1]))
for i in range(N):
    model = PGDAttack(model=gcn, nnodes=adj.shape[0], loss_type='CE', device='cpu').to('cpu')
    B = 0.01 + np.zeros((np.shape(adj)[0],np.shape(adj)[0]))
    adj_n = np.logical_xor(adj, np.random.binomial(1,B)).float()+torch.zeros((np.shape(adj)[0],np.shape(adj)[0]))
    adj_nn = torch.triu(adj_n,diagonal=1)
    adj_noise = adj_nn.T+adj_nn
    model.attack(features, adj_noise, labels, idx_train, n_perturbations=100000)
    M_adj = np.vstack((M_adj,model.modified_adj.reshape(1,np.shape(adj)[0]*np.shape(adj)[1])))

MM_adj = M_adj[1:,:]
R_adj=np.int64(sum(MM_adj)/N>0.5)
Rr_adj=R_adj.reshape(np.shape(adj)[0],np.shape(adj)[1])
np.shape(np.where((Rr_adj-np.int64(adj))!=0))[1]/(np.shape(adj)[0]*np.shape(adj)[1])#recover error

O1=gcn.predict(features, Rr_adj).argmax(1)
O2=gcn.predict(features, adj).argmax(1)
np.shape(np.where(O1!=O2))[1]#misclassification rate

In [ ]:
# DICE Attack
N=10
M_adj = np.zeros((1,np.shape(adj)[0]*np.shape(adj)[1]))
adj_mm = np.zeros((np.shape(adj)[0],np.shape(adj)[1]))
for i in range(N):
    model = DICE()
    B = 0.01 + np.zeros((np.shape(adj)[0],np.shape(adj)[0]))
    adj_n = np.logical_xor(adj, np.random.binomial(1,B)).float()+torch.zeros((np.shape(adj)[0],np.shape(adj)[0]))
    adj_nn = torch.triu(adj_n,diagonal=1)
    adj_noise = adj_nn.T+adj_nn
    adj_nnn = sp.csr_matrix(adj_noise).tocsr()
    model.attack(adj_nnn, labels, n_perturbations=100000)
    modified_adj=model.modified_adj
    for ii in range(np.shape(modified_adj)[0]):
                a = modified_adj[ii].rows           
                for jj in range(len(a[0])):
                    adj_mm[ii,a[0][jj]]=1    
    M_adj = np.vstack((M_adj,adj_mm.reshape(1,np.shape(adj)[0]*np.shape(adj)[1])))

MM_adj = M_adj[1:,:]
R_adj=np.int64(sum(MM_adj)/N>0.5)
Rr_adj=R_adj.reshape(np.shape(adj)[0],np.shape(adj)[1])
np.shape(np.where((Rr_adj-np.int64(adj))!=0))[1]/(np.shape(adj)[0]*np.shape(adj)[1])#recover error

O1=gcn.predict(features, Rr_adj).argmax(1)
O2=gcn.predict(features, adj).argmax(1)
np.shape(np.where(O1!=O2))[1]#misclassification rate

In [104]:
#Random Attack
j=1
Out_1 =np.array([])
Out_2 =np.array([])
while(j<=1000):# 1000 target nodes
    M_adj = torch.zeros((1,np.shape(adj)[0]*np.shape(adj)[1]))
    degrees = adj.sum(0)
    k = np.random.randint(0,np.shape(idx_unlabeled)[0])
    target_node = idx_unlabeled[k]
    if int(degrees[target_node])<10:
        continue
    adj_mat = np.zeros((np.shape(adj)[0],np.shape(adj)[1]))
    for i in range(10):#repreat number 10
        model = RND()
        B = 0.0001 + np.zeros((np.shape(adj)[0],np.shape(adj)[0]))
        adj_n = np.logical_xor(adj, np.random.binomial(1,B)).float()+torch.zeros((np.shape(adj)[0],np.shape(adj)[0]))
        adj_nn = torch.triu(adj_n,diagonal=1)
        adj_noise = adj_nn.T+adj_nn
        adj_nnn = sp.csr_matrix(adj_noise).tocsr()
        model.attack(adj_nnn, labels, idx_train, target_node, n_perturbations=100)
        for ii in range(np.shape(adj)[0]):
            a = model.modified_adj[ii].rows
            for jj in range(len(a[0])):
                adj_mat[ii,a[0][jj]]=1
        M_adj = np.vstack((M_adj,adj_mat.reshape(1,np.shape(adj)[0]*np.shape(adj)[1])))
    MM_adj = M_adj[1:,:]
    R_adj=np.int64(sum(MM_adj)/10>0.5)
    Rr_adj=R_adj.reshape(np.shape(adj)[0],np.shape(adj)[1])
    np.shape(np.where((Rr_adj-np.int64(adj))!=0))[1]/(np.shape(adj)[0]*np.shape(adj)[1])#recover error
    O1=gcn.predict(features, Rr_adj).argmax(1)[target_node]
    O2=gcn.predict(features, adj).argmax(1)[target_node]
    Out_1=np.hstack((Out_1,O1))
    Out_2=np.hstack((Out_2,O2))
    j+=1
np.shape(np.where(Out_1-Out_2!=0))[1]/1000 #mr


number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
620
tensor(4)
tensor(4)
tensor(4)
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
1428
tensor(4)
tensor(4)
tensor(4)
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
620
tensor(4)
tensor(4)
tensor(4)
number of pertubations: 100

number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
2084
tensor(4)
tensor(4)
tensor(4)
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
953
tensor(4)
tensor(4)
tensor(4)
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
1699
tensor(4)
tensor(4)
tensor(4)
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 1000
number of pertubations: 10

In [486]:
# train surrogate model
degrees = adj.sum(0).A1
n_perturbations = int(degrees[u]) # How many perturbations to perform. Default: Degree of the node
#model = RND()
#model.attack(adj, labels, idx_train, target_node, n_perturbations, verbose=False)


# modified_adj = model.attack(adj, labels, idx_train, target_node, n_perturbations)
model.add_nodes(features, adj, labels, idx_train, target_node, n_added=0,
                               n_perturbations=n_perturbations)
modified_adj = model.modified_adj
modified_features = model.modified_features
#modified_features = features
gcn = GCN(nfeat=features.shape[1],
                  nhid=16,
                 nclass=labels.max().item() + 1,
                  dropout=0.5, device=device)

gcn = gcn.to(device)
output = gcn.predict(modified_features, modified_adj)

print(np.shape(modified_features))
print(output)

def test(adj, features, target_node):
    ''' test on GCN '''
    gcn = GCN(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device=device)

    gcn = gcn.to(device)

    gcn.fit(features, adj, labels, idx_train)
    gcn.eval()
    output = gcn.predict()
    probs = torch.exp(output[[target_node]])[0]
    print('probs: {probs.detach().cpu().numpy()}')

    labels_tensor = torch.LongTensor(labels).to(device)
    loss_test = F.nll_loss(output[idx_test], labels_tensor[idx_test])
    acc_test = accuracy(output[idx_test], labels_tensor[idx_test])

    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

    return acc_test.item()

number of pertubations: 8
(3025, 1870)
tensor([[-2.0488, -0.5839, -1.1602],
        [-1.5482, -0.6928, -1.2476],
        [-1.6864, -0.6142, -1.2956],
        ...,
        [-1.6784, -0.7289, -1.1060],
        [-1.8741, -0.5094, -1.4039],
        [-1.2342, -0.8546, -1.2606]], grad_fn=<LogSoftmaxBackward>)


In [106]:
def single_test(adj, features, target_node, gcn=None):
    if gcn is None:
        # test on GCN (poisoning attack)
        gcn = GCN(nfeat=features.shape[1],
                  nhid=16,
                  nclass=labels.max().item() + 1,
                  dropout=0.5, device=device)

        gcn = gcn.to(device)

        gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)
        gcn.eval()
        output = gcn.predict()
    else:
        # test on GCN (evasion attack)
        output = gcn.predict(features, adj)
    probs = torch.exp(output[[target_node]])
    #print(output)
    #print(output.argmax(1))
    #print(output.argmax(1)[target_node], labels[target_node])

    # acc_test = accuracy(output[[target_node]], labels[target_node])
    acc_test = (output.argmax(1)[target_node] == labels[target_node])
    return acc_test.item()

In [107]:
def select_nodes(target_gcn=None):
    '''
    selecting nodes as reported in nettack paper:
    (i) the 10 nodes with highest margin of classification, i.e. they are clearly correctly classified,
    (ii) the 10 nodes with lowest margin (but still correctly classified) and
    (iii) 20 more nodes randomly
    '''

    if target_gcn is None:
        target_gcn = GCN(nfeat=features.shape[1],
                  nhid=16,
                  nclass=labels.max().item() + 1,
                  dropout=0.5, device=device)
        target_gcn = target_gcn.to(device)
        target_gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)
    target_gcn.eval()
    output = target_gcn.predict()

    margin_dict = {}
    for idx in idx_test:
        margin = classification_margin(output[idx], labels[idx])
        if margin < 0: # only keep the nodes correctly classified
            continue
        margin_dict[idx] = margin
    sorted_margins = sorted(margin_dict.items(), key=lambda x:x[1], reverse=True)
    high = [x for x, y in sorted_margins[: 10]]
    low = [x for x, y in sorted_margins[-10: ]]
    other = [x for x, y in sorted_margins[10: -10]]
    other = np.random.choice(other, 20, replace=False).tolist()

    return high + low + other

In [108]:
def multi_test_evasion():
    # test on 40 nodes on evasion attack
    target_gcn = GCN(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device=device)

    target_gcn = target_gcn.to(device)

    target_gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)

    cnt = 0
    degrees = adj.sum(0).A1
    node_list = select_nodes(target_gcn)
    num = len(node_list)

    print('=== [Evasion] Attacking %s nodes respectively ===' % num)
    for target_node in tqdm(node_list):
        n_perturbations = int(degrees[target_node])
        model = RND()
        model = model.to(device)
        model.attack(adj, labels, idx_train, target_node, n_perturbations, verbose=False)
        model.add_nodes(features, adj, labels, idx_train, target_node, n_added=40,
                               n_perturbations=n_perturbations)
        modified_adj = model.modified_adj
        modified_features = model.modified_features
        #print(modified_adj)
        print(modified_features)
        acc = single_test(modified_adj, modified_features, target_node, gcn=target_gcn)
        if acc == 0:
            cnt += 1
    print('misclassification rate : %s' % (cnt/num))

In [115]:
def main():
    print('=== testing GCN on original(clean) graph ===')
    test(adj, features, target_node)
    print('=== testing GCN on perturbed graph ===')
    if modified_features is None:
        test(modified_adj, features, target_node)
    else:
        test(modified_adj, modified_features, target_node)

    return modified_features
   

In [116]:

if __name__ == '__main__':
    mf = main()
   # multi_test_evasion()

=== testing GCN on original(clean) graph ===
probs: {probs.detach().cpu().numpy()}
Test set results: loss= 1.3053 accuracy= 0.7227
=== testing GCN on perturbed graph ===
probs: {probs.detach().cpu().numpy()}
Test set results: loss= 1.2632 accuracy= 0.7145
